In [438]:
from fastapi import FastAPI

import numpy as np
import pandas as pd
import uvicorn

In [ ]:
app = FastAPI()
genreFunction = pd.read_parquet('./data/genre_functions.parquet')

In [ ]:
userRecommend = pd.read_parquet('./data/recommendations.parquet')

In [ ]:
items = pd.read_parquet('./data/items.parquet')
items

In [ ]:

# We apply the str.lower() method so that it does not matter how the word is written.
genreFunction['genres'] = genreFunction['genres'].str.lower()
genreFunction['genres'].unique()

In [ ]:
genreFunction

In [ ]:
mostPlayed = genreFunction.groupby('user_id')['playtime_forever'].sum()
mostPlayed.head()

In [565]:
# The app.get decorator is used to associate the function with an HTTP GET request.
# '/PlayTimeGenre/{genre}/' is the path for the endpoint, the {genre} is the parameter.
@app.get('/PlayTimeGenre/{genre}/')
def PlayTimeGenre( genre : str ):
    '''
    Find the genre with the most playtime hours.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    int
        Year when the highest number of hours played was recorded for that genre.
    
    Examples
    -------
    >>> PlayTimeGenre('Action')
    2012
    >>> PlayTimeGenre('Indie')
    2006

    '''
    # We look in our DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]
    if genres.empty:
        return f'The genre {genre} does not exist.'

    # We calculate the total playtime for each year in the dataset.
    results = genres.groupby('year')['playtime_forever'].sum()
    
    # Locate the index where the maximum value is. In this case the most played genre.
    idResut = results.idxmax()
    
    return f'Year with the most playtime hours for {genre}: {idResut}'

In [ ]:
test = PlayTimeGenre('RPG')
print(test)

# Segunda Función

In [564]:
@app.get('/PlayTimeGenre/{genre}/')
def UserForGenre( genre : str ):
    '''
    Find the user with the most hours played by genre, as well as hours played for each year.

    Parameters
    ----------
    genre : str
        Desired genre to look for.
    
    Returns
    -------
    list
        The user name and the amount of hours played by year for that particular genre.
    
    Examples
    -------
    >>> UserForGenre('Simulation')
    UserName is the user with the most playtime for the genre "Action" with 23721 hours played.

    Year            2003    2006    2009    2010    2011    2012    2013    2014    2015    2016    
    Hours Played      0      0      2037    4102    1968     223     323    342     1224     112   
    '''
    
    
    # Look in the DataSet if there is any match with the genre entered.
    genres = genreFunction[genreFunction['genres'] == genre.lower()]

    if genres.empty:
        return f'The genre {genre} does not exist.'

    # Group the DataSet by user ID, then sum the amount of hours played.
    mostPlayed = genres.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Locate the index for the player.
    player = genres.loc[genres['playtime_forever'].idxmax()]['user_id']

    # Filter the DataSet with only the player id
    filteredDFWithPlayerID = (genres[genres['user_id'] == player])

    # Create a new DF with just the year and playtime_forever columns
    hoursPlayedByYear = filteredDFWithPlayerID.groupby('year')['playtime_forever'].sum()
    
    
    print(f'{player} is the user with the most playtime for the genre "{genre.capitalize()}" with {filteredDFWithPlayerID["playtime_forever"].sum()} hours played.')
    print(f'\nYear\tHours Played\n')
    for year, hour in hoursPlayedByYear.items():
        print(f'{year}\t{hour}\n')

In [ ]:
UserForGenre('actiON')

# 3rd Function

In [391]:
userRecommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5948940 entries, 0 to 5948939
Data columns (total 9 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   title               object
 1   id                  int32 
 2   user_id             object
 3   playtime_forever    int32 
 4   year                int16 
 5   posted              object
 6   item_id             int32 
 7   recommend           bool  
 8   sentiment_analysis  int8  
dtypes: bool(1), int16(1), int32(3), int8(1), object(3)
memory usage: 226.9+ MB


In [375]:
@app.get('/UsersRecommend/{year}/')
def UsersRecommend( year : int ):
    '''
    Get the three most recommended games  
    
    Parameters
    ----------
    year : str
        Year in which the top three recommended games are.
    
    Returns
    -------
    list
        Name of the three recommended games.
    
    Examples
    -------
    >>> UsersRecommend(2018):
    Counter-Strike: Global Offense, Garry's Mode, Fall Guys
    >>> UsersRecommend(2021)
    Empire: Total War, Left 4 Dead 2, The Stanley Parable
    '''

    # Create a DataSet with rows that match the year.
    givenYear = userRecommend[userRecommend['year'] == year]

    if givenYear.empty:
        return f'There are no records for the year {year}.'

    # Group the top three games
    topThree = (givenYear['title'].value_counts().head(3).reset_index()
        .rename(columns={ 'title': 'Game', 'count': 'Positive Reviews'}))
    

    return [{f'Top {i+1}: "{game}" with {reviews} positive reviews'} for i, (game, reviews) in topThree.iterrows()]


In [340]:
UsersRecommend(2000)

[{'Top 1: "Counter-Strike" with 10851 positive reviews'},
 {'Top 2: "Hitman: Codename 47" with 4966 positive reviews'},
 {'Top 3: "Deus Ex: Game of the Year Edition" with 4937 positive reviews'}]

# 4th Function

In [435]:
@app.get('/UsersNotRecommend/{year}/')
def UsersNotRecommend( year : int ):
    '''
    Get the three least recommended games.
    
    Parameters
    ----------
    year : str
        Year in which the three least recommended games are.
    
    Returns
    -------
    list
        Name of the three recommended games.
    
    Examples
    -------
    >>> UsersNotRecommend(2008):
    Portal 2, Garry's Mode, Fall Guys
    >>> UsersNotRecommend(2011)
    Carmageddon Max Pack, Left 4 Dead 2, The Stanley Parable
    '''

    givenYear = userRecommend[userRecommend['year'] == year]

    if givenYear.empty:
        return f'There are no records for the year {year}.'

    # Group the games that are from the desired year and had negative reviews.
    leastRecommendedGames = userRecommend[(userRecommend['year'] == year) & (userRecommend['recommend'] == False)]

    # Create a list that has the 3 least recommended games for that year.
    leastThree = (
        leastRecommendedGames['title']
        .value_counts()
        .head(3)
        .reset_index()
        .rename(columns={'count': 'Negative Reviews', 'title': 'Game'})
    )

    leastThree = [{f'Top {i+1}: "{game}" with {reviews} negative reviews'} for i, (game, reviews) in leastThree.iterrows()]

    return leastThree

In [436]:
UsersNotRecommend(2018)

[{'Top 1: "Lost Moon" with 40 negative reviews'},
 {'Top 2: "Starwalker" with 12 negative reviews'},
 {'Top 3: "Empires Of Creation" with 9 negative reviews'}]

In [437]:
userRecommend.head(1)

,title,id,user_id,playtime_forever,year,posted,item_id,recommend,sentiment_analysis
0,Carmageddon Max Pack,282010,UTNerd24,5,1997,"May 6, 2014",244210,True,0


# 5th Function

In [571]:
@app.get('/sentiment_analysis/{year}/')
def sentiment_analysis( year : int ):
    '''
    Get the category reviews from all users in a year.
    
    Parameters
    ----------
    year : str
        Desired year to see how the reviews were.
    
    Returns
    -------
    list
        Amount of all the different review categories for that year.
    
    Examples
    -------
    >>> sentiment_analysis(2018)
    {Negative = 101, Neutral = 142, Positive = 221}
    >>>sentiment_analysis(2019)
    {Negative = 140, Neutral = 47, Positive = 115}
    '''

    givenYear = userRecommend[userRecommend['year'] == year]

    if givenYear.empty:
        return f'There are no records for the year {year}.'
    
    
    sentiment = userRecommend.groupby('year')['sentiment_analysis'].value_counts().unstack(fill_value=0)


    sentiment = sentiment.loc[year].to_dict()
     
    return {"Negative": sentiment.get(0, 0),
            "Neutral": sentiment.get(1, 0),
            "Positive": sentiment.get(2, 0)}
            


In [574]:
sentiment_analysis(2009)

{'Negative': 70585, 'Neutral': 59924, 'Positive': 175569}

In [573]:
sentiment_analysis(2018)

{'Negative': 101, 'Neutral': 142, 'Positive': 221}